In [1]:
jump_server_address = '10.251.6.31'   # The internal ip Address for the Jump server
local_IP_address = '127.0.0.1'  # ip Address of the machine you are connecting from
username = "adm.davies"
password = "9kO2x@D0QB%ItfF#&UtLwyQLu3X$Bq"
IP_Address = "10.20.255.1"
interfaces = list()

In [2]:
import os
import paramiko
import textfsm
import ipaddress
import datetime as time
debug = 0
from tabulate import tabulate

In [3]:
def error_log(message,):
    date_time_object = time.datetime.now()
    datetime = date_time_object.strftime("%d/%m/%Y %H:%M:%S")
    error_file = open("Error Log.txt", "a")
    error_file.write(f"{datetime} - {message}")
    error_file.write("\n")
    error_file.close()
    if debug == 1:
        print(message)


def output_log(message,):
    date_time_object = time.datetime.now()
    datetime = date_time_object.strftime("%d/%m/%Y %H:%M:%S")
    output_file = open("Output Log.txt", "a")
    output_file.write(f"{datetime} - {message}")
    output_file.write("\n")
    output_file.close()
    if debug == 1:
        print(message)
        
def ip_check(ip):
    try:
        ipaddress.ip_address(ip)
        return True
    except ValueError:
        return False

In [4]:
def open_session(ip):
    if not ip_check(ip):
        error_log(f"open_session function error: "
                  f"ip Address {ip} is not a valid Address. Please check and restart the script!",)
        return None, False
    try:
        output_log(f"Trying to establish a connection to: {ip}")
        jump_box = paramiko.SSHClient()
        jump_box.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        jump_box.connect(jump_server_address, username=username, password=password)
        jump_box_transport = jump_box.get_transport()
        src_address = (local_IP_address, 22)
        destination_address = (ip, 22)
        jump_box_channel = jump_box_transport.open_channel("direct-tcpip", destination_address, src_address)
        target = paramiko.SSHClient()
        target.set_missing_host_key_policy(paramiko.AutoAddPolicy())
        target.connect(destination_address, username=username, password=password, sock=jump_box_channel)
        output_log(f"Connection to ip: {ip} established")
        return target, jump_box, True
    except paramiko.ssh_exception.AuthenticationException:
        error_log(f"Authentication to ip: {ip} failed! Please check your ip, username and password.")
        return None, None, False
    except paramiko.ssh_exception.NoValidConnectionsError:
        error_log(f"Unable to connect to ip: {ip}!")
        return None, None, False
    except (ConnectionError, TimeoutError):
        error_log(f"Timeout error occurred for ip: {ip}!")
        return None, None, False
    except Exception as err:
        error_log(f"Open Session Error: An unknown error occurred for ip: {ip}!")
        error_log(f"\t Error: {err}")
        return None, None, False

In [5]:
ssh, jumpbox, connection = open_session(IP_Address)
stdin, stdout, stderr = ssh.exec_command("show cdp neighbors detail")
stdout = stdout.read()
stdout = stdout.decode("utf-8")

In [16]:
with open("template.txt") as f:
    re_table = textfsm.TextFSM(f)
    header = re_table.header
    result = re_table.ParseText(stdout)
    print(tabulate(result, headers=header))

DEST_HOST                               MGMNT_IP       PLATFORM                 LOCAL_PORT             REMOTE_PORT            IOS_VERSION
--------------------------------------  -------------  -----------------------  ---------------------  ---------------------  -------------
gb-brw1-001csw001.cns.muellergroup.com  10.250.5.146   cisco WS-C4507R+E        GigabitEthernet5/46    FastEthernet1          15.2(2)E3
gb-brw1-001csw001.cns.muellergroup.com  10.250.5.146   cisco WS-C4507R+E        FastEthernet1          GigabitEthernet5/46    15.2(2)E3
gb-brw1-003asw001.cns.muellergroup.com  10.20.254.26   cisco WS-C2960X-48FPS-L  GigabitEthernet1/3     GigabitEthernet1/0/49  15.2(2)E3
gb-brw1-012asw001.cns.muellergroup.com  10.20.254.37   cisco WS-C2960X-48FPS-L  GigabitEthernet1/5     GigabitEthernet1/0/49  15.2(2)E3
gb-brw1-001vgw001.cns.muellergroup.com  10.250.5.145   Cisco CISCO3925-CHASSIS  GigabitEthernet5/4     GigabitEthernet0/0     15.2(4)M7
gb-brw1-010asw002.cns.muellergroup.com  10

In [17]:
result

[['gb-brw1-001csw001.cns.muellergroup.com',
  '10.250.5.146',
  'cisco WS-C4507R+E',
  'GigabitEthernet5/46',
  'FastEthernet1',
  '15.2(2)E3'],
 ['gb-brw1-001csw001.cns.muellergroup.com',
  '10.250.5.146',
  'cisco WS-C4507R+E',
  'FastEthernet1',
  'GigabitEthernet5/46',
  '15.2(2)E3'],
 ['gb-brw1-003asw001.cns.muellergroup.com',
  '10.20.254.26',
  'cisco WS-C2960X-48FPS-L',
  'GigabitEthernet1/3',
  'GigabitEthernet1/0/49',
  '15.2(2)E3'],
 ['gb-brw1-012asw001.cns.muellergroup.com',
  '10.20.254.37',
  'cisco WS-C2960X-48FPS-L',
  'GigabitEthernet1/5',
  'GigabitEthernet1/0/49',
  '15.2(2)E3'],
 ['gb-brw1-001vgw001.cns.muellergroup.com',
  '10.250.5.145',
  'Cisco CISCO3925-CHASSIS',
  'GigabitEthernet5/4',
  'GigabitEthernet0/0',
  '15.2(4)M7'],
 ['gb-brw1-010asw002.cns.muellergroup.com',
  '10.20.254.34',
  'cisco WS-C2960X-48FPS-L',
  'GigabitEthernet1/7',
  'GigabitEthernet1/0/49',
  '15.2(2)E3'],
 ['gb-brw1-001asw002.cns.muellergroup.com',
  '10.20.254.20',
  'cisco WS-C2960X-